# ML TFIDF WITHOUT CLASS 0

In [ ]:
from frugal.data_preproc.data import load_and_split_data , load_datasets ,make_train_test_split
from frugal.evaluation import evaluate, concatenate_evaluations , classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from datasets import load_dataset
from frugal.config.environment import DATA_DIR
import os
import pandas as pd


In [2]:
def load_and_split_data_withou_class_n(filtered_class : str):
    """
    Loads datasets from source withou a certain class "filter_class" and splits them into features and labels for training and testing.
    """
    train_dataset, test_dataset = load_datasets()

    train_dataset_filtered = train_dataset[train_dataset['label']!=filtered_class]
    test_dataset_filtered = test_dataset[test_dataset['label']!=filtered_class]
    X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered = make_train_test_split(train_dataset_filtered,test_dataset_filtered)
    return X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered


In [3]:
load_and_split_data_withou_class_n("0_not_relevant")[3].value_counts()

✅ Train and test datasets successfully loaded from Hugging Face 🤗
✅ Data successfully split into X and y for train and test sets


label
5_science_unreliable               160
4_solutions_harmful_unnecessary    160
1_not_happening                    154
6_proponents_biased                139
2_not_human                        137
3_not_bad                           97
7_fossil_fuels_needed               65
Name: count, dtype: int64

In [36]:
X_train, X_test, y_train , y_test = load_and_split_data_withou_class_n("0_not_relevant")

✅ Train and test datasets successfully loaded from Hugging Face 🤗
✅ Data successfully split into X and y for train and test sets


In [89]:
def pre_clean_data(data : pd.DataFrame):
    def clean_one_quote(quote):
        # Pre-cleaning
        text_cleaned = quote.lower()
        text_cleaned = ''.join(char for char in text_cleaned if not char.isdigit())
        """ text = quote.lower()
        text = ''.join(char for char in text if not char.isdigit())
        for punctuation in string.punctuation:
            text = text.replace(punctuation, '')
        text = text.strip() """
        return text_cleaned

    return data['quote'].map(clean_one_quote)


In [ ]:
texts = pd.DataFrame(["Alex is the best data scientist &à HAHA @@ ^^","Le Wagon 1 des 10 est &ç-!!!// SUPER"], columns=["quote"])

In [74]:
texts

,quote
0,Alex is the best data scientist &à HAHA @@ ^^
1,Le Wagon est &ç-!!!// SUPER


In [90]:
pre_clean_data(texts)

0    alex is the best data scientist &à haha @@ ^^
1                      le wagon est &ç-!!!// super
Name: quote, dtype: object

In [57]:
def tfidf(X_train, X_test, y_train) :
    # Vectorisation
    vectorizer = TfidfVectorizer()
    X_train_vec = vectorizer.fit_transform(X_train.quote)
    X_test_vec = vectorizer.transform(X_test.quote)
    # Training
    model = MultinomialNB()
    model.fit(X_train_vec, y_train)
    # Prediction
    y_pred = model.predict(X_test_vec)
    return y_pred

In [59]:
y_pred = tfidf(X_train, X_test, y_train)

In [65]:
# Get model performance
classification_report(y_test, y_pred)

/Users/datascience/.pyenv/versions/3.10.6/envs/frugal/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/datascience/.pyenv/versions/3.10.6/envs/frugal/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/datascience/.pyenv/versions/3.10.6/envs/frugal/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

'                                 precision    recall  f1-score   support\n\n                1_not_happening       0.73      0.69      0.71       154\n                    2_not_human       0.56      0.72      0.63       137\n                      3_not_bad       1.00      0.03      0.06        97\n4_solutions_harmful_unnecessary       0.57      0.82      0.67       160\n           5_science_unreliable       0.48      0.70      0.57       160\n            6_proponents_biased       0.71      0.64      0.67       139\n          7_fossil_fuels_needed       0.00      0.00      0.00        65\n\n                       accuracy                           0.59       912\n                      macro avg       0.58      0.52      0.47       912\n                   weighted avg       0.61      0.59      0.54       912\n'